<div style ="border: blue solid 5px; font-family: Agency FB;	box-shadow: 0px 0px 100px rgba(0, 0, 0, 0.5);"><center>
<a href="#" ><img src="img/python.png" style="float:left; max-width: 130px; display: inline" alt="Python"/></a> 
    <h1> <font color="blue">PROJET DATAMINIG - MACHINE LEARNING</font> </h1> 
<a href="#" ><img src="img/ensae.jpg" style="float:right; max-width: 190px; display: inline" alt="ENSAE" /></a>     
</center>
</div>

<div style ="border: blue solid 5px; font-family: Agency FB;font-size:20px;	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);background-color: rgba(10, 10, 100, 0.5);">
<center>
<font color="white">------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</font> 
<font color="white"><br><h6>--------------------------SOUS LA SUPERVISION DE : ----------------------</h6> </font> 
<ul>
<h6>M. Abdou Aziz NDIAYE</h6>
</ul>
<font color="white"><br><h6>--------------------------REALISE PAR : ----------------------</h6> </font> 
<ul>
<h6>SAMB Coumba</h6>
<h6>SOW Doulo</h6>
<h6>TRAORE Harouna</h6>
</ul>
<font color="white">------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</font> 
</center>
    </div>

<div style ="border: blue solid 5px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 50, 50, 0.5);"><font color="black"><center><h1>--1. Chargement des packages essentiels --</h1> </center></font> </div>

In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
#Transformation des variables
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#Sélection des variables
from sklearn.feature_selection import SelectKBest, f_classif
#Paramètre des modèles
from sklearn.metrics import recall_score, confusion_matrix, classification_report,roc_auc_score, roc_curve, auc

#Sélection de modèle
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV
, cross_val_score
### Pipeline
from sklearn.pipeline import Pipeline
##Pour numpy
import numpy as np
##seaborn
import seaborn as sns
##DISCRETISATION
from sklearn.preprocessing import KBinsDiscretizer
##
from outliers import smirnov_grubbs as grubbs


<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"><center><h6>--Préparation de la base --</h6></div>

In [ ]:
#Choix du repertoire de travail 
repertoire=os.chdir("C:/Users/doulo/Cours Python Data  Mining/PROJET")
print(os.getcwd())

In [ ]:
#Chargement de la base 
base=pd.read_csv('Dataset.csv',sep=';', encoding='utf-8')

In [ ]:
# Copier le data frame (Df) sur un autre objet
df=base.copy()

In [ ]:
df.head()

In [ ]:
df.shape

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"><center><h6>--Traitement préliminaire --</h6>

In [ ]:
##Modifier les indexes
#Remplacer les indexes par les noms
df.set_index('CLIENTNUM',inplace=True)

In [ ]:
#Variables categorielles
df_c=df.select_dtypes('object').head()

In [ ]:
#Variables numériques
df_n=df.select_dtypes('number').head()

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"><center><h6>--Sortie des statistiques vers Excel_CSV --</h6></center>

In [ ]:
##Statistiques descriptives pour les variables numériques
stats=df_n.describe()
stats.to_csv('Tableau_statistiques.csv', index= True, mode = 'a')

<div style ="border: blue solid 5px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 50, 50, 0.5);"><center><h1>--TRAITEMENTS DES VARIABLES --</h1></center></div>

In [ ]:
#Valeurs manquantes ?

type(df.isna().sum().sort_values(ascending=False))
df.isna().sum().sort_values(ascending=False)

In [ ]:
#1. Méthodes graphiques : Box-plot, nuage de points, histogrammes...

In [ ]:
def boxplot(dataframe):
    for colonne in dataframe.columns:
        plt.boxplot(dataframe[colonne],showmeans=True)
        print('***************************************************************************')
        print('           :------------','BOXPLOT: ', colonne,'------------:')
        print('***************************************************************************')
        plt.show()

In [ ]:
boxplot(df_n)

In [ ]:
def bplot_groupby(df_n,df_c,df):
    for y in df_n.columns:
        for x in df_c.columns:
                sns.boxplot(y=y, x=x, 
                data=df, 
                palette="colorblind",
                hue='Attrition_Flag') 
                print('***************************************************************************')
                print('           :----','BOXPLOT: ', y,'selon', x, '------------:')
                print('***************************************************************************')
                plt.show()

In [ ]:
bplot_groupby(df_n,df_c,df)

In [ ]:
var_aberr=["Months_on_book", "Months_Inactive_12_mon", 
           "Total_Revolving_Bal","Total_Trans_Amt",
           "Total_Ct_Chng_Q4_Q1","Total_Ct_Chng_Q4_Q1",
           "Avg_Utilization_Ratio"]

In [ ]:
grouped = df.groupby('Attrition_Flag')

In [ ]:
statBefore = pd.DataFrame({'q1': grouped['Customer_Age'].quantile(.25), \
'median': grouped['Customer_Age'].median(), 'q3' : grouped['Customer_Age'].quantile(.75)})

In [ ]:
def is_outlier(row):
       iq_range = statBefore.loc[row.Attrition_Flag]['q3'] - statBefore.loc[row.Attrition_Flag]['q1']
       median = statBefore.loc[row.Attrition_Flag]['median']
       if row.Customer_Age > (median + (1.5* iq_range)) or row.Customer_Age < (median - (1.5* iq_range)):
           return True
       else:
           return False
#apply the function to the original df:
df.loc[:, 'outlier'] = df.apply(is_outlier, axis = 1)
#filter to only non-outliers:
df_no_outliers = df[~(df.outlier)]

In [ ]:
df_no_outliers.shape

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6>VALEURS ABERRANTES</h6></center>

In [ ]:
grubbs.max_test_outliers(np.array(df["Total_Ct_Chng_Q4_Q1"]), alpha=.05)

In [ ]:
index_out =grubbs.max_test_indices(np.array(df["Total_Ct_Chng_Q4_Q1"]), alpha=.05)
len(index_out)


In [ ]:
index_out

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6>FONCTIONS DE SUPPRESSION DES OUTLIERS

In [ ]:
df=base.copy()

In [ ]:
## Drop outliers à gauche
def drop_outlier_right(variable):
    index_out =grubbs.max_test_indices(np.array(df[variable]), alpha=.05)
    print('****************************', variable ,'******************************************')
    print('Nombre de valeurs aberrantes à droite  : ', len(index_out),'-------------:')
    print('***************************************************************************')
    accord=int(input("Tapez 1 pour confirmer la suppression"))
    if (accord==1):
        df.drop(list(index_out),0, inplace=True)
        print('Valeurs supprimées')
    else : 
        print('Annuler') 

In [ ]:
for var in var_aberr:
    drop_outlier_right(var)

In [ ]:
## Drop outliers à droite
def drop_outlier_left(variable):
    index_out =grubbs.min_test_indices(np.array(df[variable]), alpha=.05)
    print('****************************', variable ,'******************************************')
    print('Nombre de valeurs aberrantes à droite  : ', len(index_out),'-------------:')
    print('***************************************************************************')
    accord=int(input("Tapez 1 pour confirmer la suppression"))
    if (accord==1):
        df.drop(index_out,0, inplace=True)
        print('Valeurs supprimées')
    else : 
        print('Annuler') 

In [ ]:
for var in var_aberr:
    drop_outlier_left(var)

In [ ]:
df.shape

In [ ]:
def tab(col):
    for colonne in df.select_dtypes('object').columns:
        print('***************************************************************************')
        print('           :------------','Attrition selon : ', colonne,'------------:')
        print('***************************************************************************')
        tab_croise=pd.crosstab(df["Attrition_Flag"],df[colonne])
        tab_croise.to_csv('Tableau_croisees.csv', index= True, mode = 'a')
        print(pd.crosstab(df["Attrition_Flag"],df[colonne]))

In [ ]:
tab(df)

In [ ]:
#Afficher une liste 
print(df.duplicated()) 

<div style ="border: blue solid 5px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 50, 50, 0.5);"><font color="black"><center><h1>ANALYSE DESCRIPTIVE</h1></center></font></div>

In [ ]:
#Catégorisation
for colonne in df.select_dtypes(['object',"category"]).columns:
    df[colonne]=df[colonne].astype('category').cat.codes

In [ ]:
#############DISCRETISATION (Pour regression logistique)
df.loc[:, 'AGE']=pd.cut(df["Customer_Age"],3,labels=["Jeune", "Adulte","Vieux"])

In [ ]:
#histogramme
def graph(data):
    for colonne in data.columns:
        print('***************************************************************************')
        print('             :------------',colonne,'------------:')
        print('***************************************************************************')
        df[colonne].hist()
        plt.show()

In [ ]:
graph(df_n)

In [ ]:
###Corrélation_Matrice
corrMatrix = df_n.corr()
sns.heatmap(corrMatrix, annot=True)
fig = plt.gcf()
fig.set_size_inches(10, 6)
plt.savefig('Matrice_Correlation.png')

<div style ="border: blue solid 5px; font-family: Agency FB; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 50, 50, 0.5);"><center><h1>ANALYSE DESCRIPTIVE avec Pandas_Profiling</h1></center></div>


In [ ]:
!pip install pandas_profiling


In [ ]:
## Statistiques descriptives
from pandas_profiling import ProfileReport
rapport=ProfileReport(base, title='Statistiques descriptives', explorative=True)
rapport.to_widgets()
rapport.to_notebook_iframe()
rapport.to_file("StatistiquesDescriptives_Op2.html")

<div style ="border: blue solid 5px; font-family: Agency FB; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 50, 50, 0.5);"><center><h1>MODELISATION</h1></center></div>


<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Importations des modèles</h1>    
</center>

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h3>1. Modèles standards</h3>    
</center>

In [ ]:
#Logistique
from sklearn.linear_model import LogisticRegression
#Arbre de decision
from sklearn.tree import DecisionTreeClassifier
#KNN
from sklearn.neighbors import KNeighborsClassifier
#SVM
from sklearn.svm import SVC



<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h3>2. Methodes ensemblistes</h3>    
</center>

In [ ]:
!pip install xgboost

In [ ]:
##########################Méthode ensembliste######################################################
#Bagging
from sklearn.ensemble import BaggingClassifier
#randomized decision trees
from sklearn.ensemble import RandomForestClassifier
#Boosting
#Adaboost
from sklearn.ensemble import AdaBoostClassifier
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
#XGBoost
from xgboost import XGBClassifier

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h3>3. Reseau de neurones</h3>    
</center>

In [ ]:
##############################  Reseau de neurones     ##############################################
#MultiLayerPerceptron
from sklearn.neural_network import MLPClassifier

<div style ="border: blue solid 3px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h3>Préparation</h3>    
</center>

In [ ]:
#Choix de la variables à expliquer
y = df['Attrition_Flag']
x = df.drop(['Attrition_Flag','AGE'], axis = 1)


In [ ]:
#Division de la base en test 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Fonction d'évaluation des modèles</h1>    
</center>

In [ ]:
#Fonction d'évaluation des modèles
def evaluation_modele(modele):
    #Modèle et paramètre
    modele.fit(x_train, y_train)
    y_pred = modele.predict(x_test)
    print("**********************************")
    print("-----------------------------------")
    print("Matrice de confusion:")
    print("-----------------------------------")
    print(confusion_matrix(y_test, y_pred))
    print("**********************************")
    print("-----------------------------------")
    print("Classification report: ")
    print("-----------------------------------")
    # Exporter les sorties vers un fichier CSV
    report=classification_report(y_test, y_pred,output_dict=True)
    report=pd.DataFrame(report).transpose()
    report.to_csv('Sorties_Modeles.csv', index= True, mode = 'a')
    print(classification_report(y_test, y_pred))
    ##Courbe d'apprentissage
    print("**********************************")
    print("-----------------------------------")
    print("Courbe d'apprentissage ")
    print("-----------------------------------")
    N, train_score, val_score = learning_curve(modele, x_train, y_train, cv=4, 
    scoring='accuracy', train_sizes=np.linspace(0.1,1,5))
    plt.figure()
    plt.plot(N, train_score.mean(axis =1), label = 'train')
    plt.plot(N, val_score.mean(axis =1), label = 'validation')
    plt.legend()
    plt.show()
    print("**********************************")
    print("-----------------------------------")
    print("Courbe de ROC ")
    print("-----------------------------------")
    ##Courbe de ROC
    fpr,tpr,_=roc_curve(y_test,y_pred,pos_label=1)
    aire=auc(fpr,tpr)
    plt.figure()
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr,tpr)
    plt.xlabel('Taux de Faux positif')
    plt.ylabel('Taux de vrai positif')
    plt.title('Courbe de ROC')
    plt.show()
    ## Affichage de la valeur de l'aire de la courbe ROC
    print("-----------------------------------")
    print("Aire en dessous de la courbe:%0.3f" %aire)
    print("-----------------------------------")
    print("**********************************")


<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Instanciation des modèles</h1>    
</center>

In [ ]:
#Instanciation des modèles sans transformation 
#Regression logistique
lr  = LogisticRegression(solver='liblinear',random_state=0,penalty='l2')
#Arbre de decision
dtc = DecisionTreeClassifier(random_state=0)
#K plus proche voisin
knn = KNeighborsClassifier()
#SVM à noyau linéaire
clf = SVC() 


In [ ]:
#Reseau de neurones 
neural= MLPClassifier(hidden_layer_sizes=(150,100,50),
                          max_iter=300,activation = 'relu',solver='adam',
                      random_state=1)

In [ ]:
#Bagging
bc = BaggingClassifier(dtc, 10, random_state=0)
#Randomized decision tree
rfc= RandomForestClassifier(max_depth=2, random_state=0)
#Adaboost
abc = AdaBoostClassifier(dtc,n_estimators=50,learning_rate=1)
#XGBClassifier
xgb=XGBClassifier(base_score= 0.5, booster='gbtree', colsample_bylevel= 1,
                  colsample_bytree=1, gamma= 0, learning_rate= 0.1, max_delta_step= 0, 
                  max_depth= 3, min_child_weight= 1, missing= None, n_estimators= 100,
                  objective='binary:logistic', reg_alpha= 0, reg_lambda=1,
                  scale_pos_weight= 1, seed= 0, silent=True, subsample=1, verbosity=0,use_label_encoder=False)


<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Regroupement des modèles</h1>    
</center>

In [ ]:
#Dictionnaire des modèles
dico_modeles = {'Régresssion logistique': lr,
                'Arbre de décision': dtc}

In [ ]:
dico1={'K plus proches voisins': knn,
                'Support Vecteur Machine':clf
               }

In [ ]:
dico_ens={'Randomized decision tree':rfc,'Adaboost':abc}

In [ ]:
dico2={'Bagging':bc,'XGBClassifier':xgb}

In [ ]:
dico_reseau={'Reseau de neurone':neural}

<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Evaluation des modèles</h1>    
</center>

In [ ]:
#Evaluation de la regression logistique et des arbres de décisions
for key, value in dico_modeles.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du KNN et du SVM
for key, value in dico1.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du bagging et de XGboost  
for key, value in dico2.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du reseau de neurone 
for key, value in dico_reseau.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du Randomized decision tree et de Adaaboost
for key, value in dico_ens.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Optimisation:Selection  et transformation des variables</h1>    
</center>

<div style ="border: blue solid 1px; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h3>CrossValidation et Grid search.</h3>    
</center>

In [ ]:
##Theorie sur les modèles 

In [ ]:
## Normalisation des données 
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
#Selection des variables les plus pertinentes pour expliquer l'attrition
skb = SelectKBest(f_classif, k=5)
x_train = skb.fit_transform(x_train, y_train)
print("----------------- Les Variables retenues sont:------------------")
#text format
df.columns[skb.get_support(indices=True)]
#vector format
variable_retenues= list(df.columns[skb.get_support(indices=True)])
print(variable_retenues)
x_test = skb.transform(x_test)

<div style ="border: blue solid 3px; font-family: Agency FB; 	box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 50, 0.5);"> <center><h6><center>
<h1>Reévalution des modèles avec moins de variables et données normalisées</h1>    
</center>

In [ ]:
#Evaluation de la regression logistique et des arbres de décisions
for key, value in dico_modeles.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du KNN et du SVM
for key, value in dico1.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du bagging et de XGboost 
for key, value in dico2.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du Randomized decision tree et de Adaaboost
for key, value in dico_ens.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

In [ ]:
#Evaluation du reseau de neurone 
for key, value in dico_reseau.items():
    print("-----------------",key,"-----------------")
    evaluation_modele(value)

<center><div style ="border: blue solid 5px; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 50, 50, 0.5);"> <h3>Arbre pour le modèle XGBosst</h3></div></center>

In [ ]:
## Installer le package graphviz
!pip install graphviz
## telecharger et Installer l'excutable de graphiz sur l'ordinateur
## Ajouter du chemin de graphique à la variable d'environnement
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'



In [ ]:
#XGBClassifier
xgb=XGBClassifier(base_score= 0.5, booster='gbtree', colsample_bylevel= 1,
                  colsample_bytree=1, gamma= 0, learning_rate= 0.1, max_delta_step= 0, 
                  max_depth= 3, min_child_weight= 1, missing= None, n_estimators= 100,
                  objective='binary:logistic', reg_alpha= 0, reg_lambda=1,
                  scale_pos_weight= 1, seed= 0, silent=True, subsample=1, verbosity=0,use_label_encoder=False)

xgb.fit(x_train, y_train)
from xgboost import plot_tree
plt.figure()
plot_tree(xgb,num_trees=5)
fig = plt.gcf()
fig.set_size_inches(15, 35)
plt.savefig('arbre.png')
plt.show()

In [ ]:
# feature importance
from xgboost import plot_importance
print(xgb.feature_importances_)
# plot
#plt.figure()
#plt.bar(range(len(xgb.feature_importances_)), xgb.feature_importances_)
plot_importance(xgb)
plt.savefig('importance_variables.png')
plt.show()

<center><div style ="border: blue solid 5px; font-family: Agency FB; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(0, 50, 250, 0.5);"><center>
<h1>Optimisation du modèle retenu</h1>    
</center></div>

In [ ]:
# Grille de paramètres à optimiser
parametres = {
        'num_boost_round': [100, 250, 500],
        'eta': [0.05, 0.1, 0.3],
        'max_depth': [6, 9, 12],
        'subsample': [0.9, 1.0],
        'colsample_bytree': [0.9, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate':[0.02,0.1],
        'n_estimators':[100,200,300],
         'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5]
        
    }
  




In [ ]:
xgb = XGBClassifier(objective='binary:logistic',silent=True,missing= None,reg_alpha= 0, reg_lambda=1,
                    verbosity=0,use_label_encoder=False,seed= 0)

In [ ]:
 xgb=GridSearchCV(xgb, parametres, n_jobs=1, cv=3)

In [ ]:
evaluation_modele(xgb)

In [ ]:
#Afficher l'arbre de décision
#tree.plot_tree(clf) 

<center><div style ="border: red solid 2px; font-family: Agency FB; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.5);	background-color: rgba(50, 0, 0, 0.5);"><center>
<h1>FIN</h1>    
</center></div>